# 1. Read the course schedule into a DataFrame

In [1]:
from bs4 import BeautifulSoup

# Load the HTML file
with open('course_schedule.html', 'r') as file:
    html_content = file.read()
    
# Create a BeautifulSoup object
dom = BeautifulSoup(html_content, "html.parser")

### 1. Extract Number-Section information for each course

Let's check how many total courses we have:

In [2]:
all_li = dom.select('.schedule-listing > .row')

In [3]:
len(all_li)

175

Let's now extract all the section numbers:

In [4]:
my_a = dom.select('.schedule-listing div > span')

In [5]:
import re

section_names = list()

for a in my_a:
    text = a.get_text().strip()
    
    #search for the pattern that starts with 'CSCI-GA...' or 'CSCI-UA'
    #some courses may have more than one course number, in that case, we will get just the first one (i.e.the main one that starts with 'CSCI...' 
    section = re.search('CSCI-[G,U]A\.\d{4}\-.+',text)
    if section:
        #get rid of the invizible space character
        section_names.append(section[0].replace('\u200b', ''))
        

In [6]:
len(section_names)

175

- We can see that the number of rows in the ul.class_schedule which is equivalent to the total number of courses matches the length of the sections list we got, meaning that we extracted the sections numbers of each course succesfully
- Some courses may have more than one course number. Usually, the second course numbers indicate that that particular course can be double-counted as the course from other department, and they have a different pattern than 'CSCI...'. Therfore, since we're focusing only on CS department's courses, we will get just the first course number (i.e. the one that starts with 'CSCI...') because that should be the the main one.  
 

### 2. Using a similar logic, extract other columns 

a. Extract course names

In [7]:
span_courses = dom.select('.schedule-listing > li span:nth-child(2)')

In [8]:
course_names = list()

for name in span_courses:
    course_name = name.get_text().strip()
    
    #replace trailing newline character and space characters in between strings with a space
    course_name = re.sub(r'\n\s+', ' ', course_name)
    course_names.append(course_name)
    

In [9]:
len(course_names)

175

- Some course names have trailing newline character and space characters in between strings, we will replace them with a space

b. Extract instructor names

In [10]:
span_instructors = dom.select('.schedule-listing > li span:nth-child(3)')

In [11]:
instructor_names = list()

for instructor in span_instructors:
    instructor_name = instructor.get_text().strip()
    
    instructor_name = instructor_name.strip().replace('\n', ';')
    
    #if the course has 2+ instructor names, we will concatenate the names in one string using comma as a separator after each full name
    if ';' in instructor_name:
        
        #replace ; with empty space
        instructor_name = re.sub(r'[; ]+', ' ', instructor_name).strip()
        
        #split the string into a list of names
        names_list = instructor_name.split()

        #create a list with full names
        full_names = [f"{names_list[i]} {names_list[i + 1]}" for i in range(0, len(names_list), 2)]

        #join the full names with commas
        result_string = ', '.join(full_names)
        instructor_names.append(result_string) 
        
    else:
        instructor_names.append(instructor_name) 
        

In [12]:
len(instructor_names)

175

- Some courses have more than one instructors, in that case, we will concatenate the names in one string using comma as a separator after each full name. We would want all the entries in the column 'instructor name' of the dataframe later to beof the same type, so instead of making a of strings for 2+ isntructor names, adding all the isntructors in one string sepprarated by comma could be more efficient. Moreover, it could also later facilitate the process of finding the number of courses each instructor teaches, for example.

c. Extract meeting times

In [13]:
span_times = dom.select('.schedule-listing > li span:nth-child(4)')

In [14]:
meeting_times = list()

for time in span_times:
    meeting_time = time.get_text().strip()
    meeting_times.append(meeting_time)
    

In [15]:
len(meeting_times)

175

### 3. Now, let's create a DataFrame object

In [25]:
import pandas as pd


# the list 'data' contains number-section, course names, instructors, and times in this order, respectively
data = zip(section_names,course_names,instructor_names,meeting_times)
columns = ['Number-Section','Name','Instructor','Time']

In [26]:
df = pd.DataFrame(data, columns = columns)

In [27]:
df

,Number-Section,Name,Instructor,Time
0,CSCI-GA.1133-001,PAC I*,CANCELLED,-
1,CSCI-GA.1133-002,PAC I Recitation,CANCELLED,-
2,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Peter Hall,R 5:55-6:45PM
4,CSCI-GA.1170-003,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM
...,...,...,...,...
170,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM
171,CSCI-UA.0480-073,Special Topics: Introduction to Robot Intellig...,Lerrel Pinto,TR 3:30-4:45PM
172,CSCI-UA.0520-001,Undergraduate Research,STAFF,-
173,CSCI-UA.0897-001,Internship,STAFF,-


**a. Break apart the Number-Section column into two separate columns: Number and Section**

In [28]:
section_separated = df['Number-Section'].str.extract('(CSCI\-\w{2}.+)-(\d+)')

In [29]:
section_separated

,0,1
0,CSCI-GA.1133,001
1,CSCI-GA.1133,002
2,CSCI-GA.1170,001
3,CSCI-GA.1170,002
4,CSCI-GA.1170,003
...,...,...
170,CSCI-UA.0480,069
171,CSCI-UA.0480,073
172,CSCI-UA.0520,001
173,CSCI-UA.0897,001


In [30]:
#delete the Number-Section column as we no longer need it (Number and Secgion columns contain the same information now)
del df['Number-Section']

In [31]:
#add the new columns to the dataframe df
df[['Number', 'Section']] = section_separated

In [32]:
df

,Name,Instructor,Time,Number,Section
0,PAC I*,CANCELLED,-,CSCI-GA.1133,001
1,PAC I Recitation,CANCELLED,-,CSCI-GA.1133,002
2,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,CSCI-GA.1170,001
3,Fundamental Algorithms Recitation,Peter Hall,R 5:55-6:45PM,CSCI-GA.1170,002
4,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM,CSCI-GA.1170,003
...,...,...,...,...,...
170,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA.0480,069
171,Special Topics: Introduction to Robot Intellig...,Lerrel Pinto,TR 3:30-4:45PM,CSCI-UA.0480,073
172,Undergraduate Research,STAFF,-,CSCI-UA.0520,001
173,Internship,STAFF,-,CSCI-UA.0897,001


**b. Show some dataframe information**

In [33]:
#show info about the data types and counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        175 non-null    object
 1   Instructor  175 non-null    object
 2   Time        175 non-null    object
 3   Number      175 non-null    object
 4   Section     175 non-null    object
dtypes: object(5)
memory usage: 7.0+ KB


In [34]:
#show the first 5 rows
df.head(5)

,Name,Instructor,Time,Number,Section
0,PAC I*,CANCELLED,-,CSCI-GA.1133,001
1,PAC I Recitation,CANCELLED,-,CSCI-GA.1133,002
2,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,CSCI-GA.1170,001
3,Fundamental Algorithms Recitation,Peter Hall,R 5:55-6:45PM,CSCI-GA.1170,002
4,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM,CSCI-GA.1170,003


In [35]:
#show the last 5 rows
df.tail(5)

,Name,Instructor,Time,Number,Section
170,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA.0480,069
171,Special Topics: Introduction to Robot Intellig...,Lerrel Pinto,TR 3:30-4:45PM,CSCI-UA.0480,073
172,Undergraduate Research,STAFF,-,CSCI-UA.0520,001
173,Internship,STAFF,-,CSCI-UA.0897,001
174,Independent Study,STAFF,-,CSCI-UA.0997,001


In [36]:
#show a random sampling of 5 rows
df.sample(5)

,Name,Instructor,Time,Number,Section
149,Programming Tools for the Data Scientist,Hasan Aljabbouli,TR 3:30-4:45PM,CSCI-UA.0381,001
53,Special Topics: Quantum Computing,Nicholas Spooner,M 2:00-4:00PM,CSCI-GA.3033,103
142,Basic Algorithms,Alan Siegel,MW 2:00-3:15PM,CSCI-UA.0310,003
132,Computer Systems Organization,Hasan Aljabbouli,TR 9:30-10:45AM,CSCI-UA.0201,003
165,Special Topics: Parallel Computing,Mohamed Zahran,TR 2:00-3:15PM,CSCI-UA.0480,051


# 2. Read the course catalog into a DataFrame

In [37]:
from bs4 import BeautifulSoup

# Load the HTML file
with open('course_catalog.html', 'r') as f:
    html_content_second = f.read()
    
# Create a BeautifulSoup object
dom_second = BeautifulSoup(html_content_second, "html.parser")

Let's check how many total courses we have listed:

In [38]:
all_li = dom_second.select('.courses-listing > li')

In [39]:
len(all_li)

96

**a. Extract Course number**

In [40]:
p_courses = dom_second.select('.courses-listing > li > p:nth-child(1)')

In [41]:
len(p_courses)

96

In [42]:
import re

catalog_numbers = list()

for course in p_courses:
    whole_str = course.get_text().strip()
    course_number = re.search('^\w+\-\w{2}\.\d{4}',whole_str)
    catalog_numbers.append(course_number[0])
    

In [43]:
len(catalog_numbers)

96

**b. Extract course prerequisites**

In [44]:
p_prereqs = dom_second.select('.courses-listing > li > p:nth-child(3)')

In [45]:
len(p_prereqs)

96

In [46]:
catalog_prereqs = list()

for course in p_prereqs:
    whole_str = course.get_text().strip()
    #each string starts with 'Prerequisites: ', we need to get a substring that comes after it
    course_prereqs = whole_str[15:]
    catalog_prereqs.append(course_prereqs)
    

In [47]:
len(catalog_prereqs)

96

**c. Extract course points**

In [48]:
p_points = dom_second.select('.courses-listing > li > p:nth-child(2)')

In [49]:
len(p_points)

96

In [50]:
catalog_points = list()
ctr = 0

for course in p_points:
    whole_str = course.get_text().strip()
    #there are different patterns in which points are listed, so we will get a string that ends with 'Points', 
    #and then extract a substring from there that corresponds to the number of points
    course_point = re.search('^.*Points',whole_str)
    course_point_upd = course_point[0][:-7]    
    catalog_points.append(course_point_upd)
    

In [51]:
len(catalog_points)

96

**d. Create a data frame**

In [52]:
import pandas as pd

# the list 'data' contains course number, prerequisites, and points in this order, respectively
data_catalog = zip(catalog_numbers,catalog_prereqs,catalog_points)
columns_catalog = ['Number','Prereqs','Points']

df_catalog = pd.DataFrame(data_catalog, columns = columns_catalog)

In [53]:
df_catalog

,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3
...,...,...,...
91,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
92,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
93,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
94,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4


**c. Show some dataframe information**

In [54]:
#show info about the data types and counts
df_catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   96 non-null     object
 1   Prereqs  96 non-null     object
 2   Points   96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [55]:
#show the first 5 rows
df_catalog.head(5)

,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3


In [56]:
#show the last 5 rows
df_catalog.tail(5)

,Number,Prereqs,Points
91,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
92,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
93,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
94,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4
95,FRSEM-UA.0597,"Some programming experience in Python, Java, J...",4


In [57]:
#show a random sampling of 5 rows
df_catalog.sample(5)

,Number,Prereqs,Points
18,CSCI-GA.2433,None,3
54,CSCI-GA.3520,Permission of the instructor for master’s stud...,4
40,CSCI-GA.2840,None,3
62,CORE-UA.0109,Department Permission,4
76,CSCI-UA.0421,"Computer Systems Organization (CSCI-UA 201), e...",4


# 3. Put together both DataFrames

In [58]:
# we need to keep all the rows from df because course catalog has some courses that are 
# not listed in the course schedule for this semester, and we need to show show all scheduled classes for this semester
new_df = pd.merge(df, df_catalog, on='Number', how = 'left')

new_cols = ["Number","Name","Instructor","Time","Prereqs","Points"]
new_df = new_df.reindex(columns=new_cols)

In [59]:
new_df

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA.1133,PAC I*,CANCELLED,-,None,4
1,CSCI-GA.1133,PAC I Recitation,CANCELLED,-,None,4
2,CSCI-GA.1170,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,At least one year of experience with a high-le...,3
3,CSCI-GA.1170,Fundamental Algorithms Recitation,Peter Hall,R 5:55-6:45PM,At least one year of experience with a high-le...,3
4,CSCI-GA.1170,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM,At least one year of experience with a high-le...,3
...,...,...,...,...,...,...
170,CSCI-UA.0480,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,Topics determine prerequisites.,4
171,CSCI-UA.0480,Special Topics: Introduction to Robot Intellig...,Lerrel Pinto,TR 3:30-4:45PM,Topics determine prerequisites.,4
172,CSCI-UA.0520,Undergraduate Research,STAFF,-,Permission of the department.,4
173,CSCI-UA.0897,Internship,STAFF,-,Restricted to declared computer science majors...,1 - 4


# 4. Conclusion

When working with the Course Schedule, there were few anomalies with the data that are as follows:

- Some courses had more than one course number. Usually, the second course numbers indicate that that particular course can be double-counted as the course from other department, and they have a different pattern than 'CSCI...'; To deal with it, I extracted only the main course number that corresponded to CSCI pattern.
- Some course names had trailing newline characters and space characters in between strings; To fix it, I replaced all of them with a space.
- Some courses had more than one instructors; To deal with it, I concatenated all the names in one string using comma as a separator after each full name. 
- Some courses had missing values for meeting times which was indicated with a dash '-' in the website, I left them as it is.

When working with the Course Catalog, the anomalies I encountered were:

- There were few courses that didn't have CSCI markings on the course number, so I adjusted my searching criteria accordingly to include all the courses.
- There were a lot of different patterns in which points were listed: some had only 1 point, some - a range of points, some - different points based on which degree the person is studying for, and so on. To deal with it, I extracted a string that ends with 'Points', and then extracted a substring from it that corresponded to the number of points.

Setting `how` to `left` when merging dataframes includes all keys from the first (left) DataFrame, even if they don't exist in second. I decided to include all the keys from the dataframe `df` obtained from reading the course schedule because the course catalog has some courses that are not listed in the course schedule for this semester. If you change the dataframes' order such that dataframe `df_catalog` will be on the left, the resulting dataframe obtained will contain 213 rows which doesn't match with the number of courses offered in the course schedule for this semester.